# Imports

In [1]:
import numpy as np
from scipy.optimize import fsolve

# Reference

Mukazhanov, O., et al. "Numerical parametrization of stationary axisymmetric black holes in a theory agnostic framework." *Phys. Rev. D* 110, 024060 https://doi.org/10.1103/PhysRevD.110.024060

In [2]:
class KRZParametrizer :
    """
    Computes Pade-approximated parameters for a black hole metric given a grid,
    enabling analytic modeling from numerical data.
    """
    
    def __init__(self, g, a, r0, rs, ths, ms, ns) :
        """
        Parameters:
        - g: dict of 2D arrays for metric components, e.g., g["gtp"], g["thth"]
        - a: spin parameter
        - r0: horizon radius
        - rs: 1D array of radial coordinates (Boyer-Lindquist)
        - ths: 1D array of angular coordinates (radians)
        - ms: dict of cosine orders "m" for each function
        - ns: dict of Pade orders "n" for each function
        """
        self.g   = g
        self.a   = a
        self.r0  = r0
        self.rs  = rs
        self.xs  = 1 - r0/rs
        self.ths = ths
        self.ys  = np.cos(ths)**2
        self.ms  = ms
        self.ns  = ns

    def helper_F_i(self, fis, x, tildeF, n) :
        k = 1
        for i in range(n-1, 0, -1) :
            gi = 1 + fis[i]*x/k
            k = gi
        return fis[0]/k*np.power(1-x, 3) - tildeF

    def helper_F(self, fis, xis, tildeFis, n) :
        return [self.helper_F_i(fis, xis[i], tildeFis[i], n) for i in range(n)]

    def helper_Aj_i(self, ais, x, tildeA, n) :
        k = 1
        for i in range(n-1, 0, -1) :
            gi = 1 + ais[i]*x/k
            k  = gi
        res = ais[0]/k*(1-x)**4
        return res - tildeA
    
    def helper_Aj(self, ais, xis, tildeAis, n) :
        return [self.helper_Aj_i(ais, xis[i], tildeAis[i], n) for i in range(n)]

    def helper_B_i(self, bis, x, tildeB, n) :
        k = 1
        for i in range(n-1, 0, -1) :
            gi = 1 + bis[i]*x/k
            k  = gi
        res = bis[0]/k*(1-x)**2
        return res - tildeB
    
    def helper_B(self, bis, xis, tildeBis, n) :
        return [self.helper_B_i(bis, xis[i], tildeBis[i], n) for i in range(n)]
        
    def Sig(self, r, th) :
        return 1 + (self.a*np.cos(th)/r)**2

    def EW(self, r_ind) :
        r      = self.rs[r_ind]
        th_pow = np.sin(self.ths)**2
        denom  = r * th_pow
        return -self.g["gtp"][r_ind] * Sig(r, self.ths) / denom

    def EW_Pade(self, x, j) :
        k = 1
        for i in range(self.ns["W"]-1, 0, -1) :
            gi = 1 + self.W_params[i+1, j]*x/k
            k = gi
        return self.W_params[1, j]/k

    def EW_num(self, x, th) :
        res = 0
        for j in range(self.ms["W"]+1) :
            res = res + np.power(np.cos(th), 2*j)*(self.EW_Pade(x, j)*np.power(1-x, 3) + self.W_params[0, j]*(1-x)**2)
        return res

    def W_num(self, x, th) :
        r = self.r0 / (1-x)
        return self.EW_num(x, th) / self.Sig(r, th)
    
    def EK(self, r_ind) :
        r_pow  = self.rs[r_ind]**2
        th_pow = np.sin(self.ths)**2
        denom  = r_pow * th_pow
        r = self.rs[r_ind]
        x = self.xs[r_ind]
        return (self.g["gpp"][r_ind]/denom - 1 - self.a*self.W_num(x, self.ths)/r) * self.Sig(r, self.ths)

    def EK_Pade(self, x, j) :
        k = 1
        for i in range(self.ns["K"]-1, 0, -1) :
            gi = 1 + self.K_params[i+1, j]*x/k
            k = gi
        return self.K_params[1, j]/k
    
    def EK_num(self, x, th) :
        res = 0
        for j in range(self.ms["K"]+1) :
            res = res + np.power(np.cos(th), 2*j)*(self.EK_Pade(x, j)*np.power(1-x, 3) + self.K_params[0, j]*(1-x)**2)
        return res
    
    def K_num(self, x, th) :
        r = self.r0 / (1-x)
        return self.EK_num(x, th)/self.Sig(r, th) + 1 + self.a*self.W_num(x, th)/r
    
    def A(self, r_ind) :
        x = self.xs[r_ind]
        return -self.g["gtt"][r_ind]*self.K_num(x, self.ths) + (self.W_num(x, self.ths)*np.sin(self.ths))**2

    def A_Pade(self, x, j) :
        k = 1
        for i in range(self.ns["A"]-1, 0, -1) :
            gi = 1 + self.A_params[i+2, j]*x/k
            k = gi
        return self.A_params[2, j]/k
    
    def A_num(self, x, th) :
        eps0 = self.A_params[0, 0]
        a00  = self.A_params[1, 0]
        k00  = self.K_params[0, 0]
        res = x*(1 - eps0*(1-x) + (a00-eps0+k00)*(1-x)**2 + self.A_Pade(x, 0)*(1-x)**3)
        for j in range(1, self.ms["A"]+1) :
            if j >= self.ms["K"]+1 :
                Kcoef = 0
            else :
                Kcoef = self.EK_Pade(x, j)*np.power(1-x, 3) + self.K_params[0, j]*(1-x)**2
            Ccoef = Kcoef + self.A_params[0, j]*(1-x)**2 + self.A_params[1, j]*(1-x)**3 + self.A_Pade(x, j)*(1-x)**4
            res = res + np.power(np.cos(th), 2*j)*Ccoef
        return res
    
    def B2(self, r_ind) :
        r = self.rs[r_ind]
        x = self.xs[r_ind]
        return self.g["grr"][r_ind]*self.A_num(x, self.ths)/self.Sig(r, self.ths)

    def compute_W_params(self) :
        # Far Field
        xs = self.xs
        ys = self.ys
        nW = self.ns["W"]
        mW = self.ms["W"]
    
        EW_far = np.empty([2, mW+1])
        for i, r_ind in enumerate((-2, -1)) :
            temp_fit = self.EW(r_ind)
            EW_far[i] = np.flip(np.polyfit(ys, temp_fit, mW))

        x1, x2 = xs[-2], xs[-1]
        p1, p2 = EW_far[0], EW_far[1]
        tildeW = -(p2 / (1 - x2)**2 - p1 / (1 - x1)**2) / (x2 - x1)
        w0     = p1 / (1 - x1)**2 - tildeW*(1 - x1)

        # Near Field
        wis_params = np.zeros([nW, mW+1])

        for nW_temp in range(1, nW+1) :
            xs_near = self.xs[:nW_temp]
            EW_near = np.empty([nW_temp, mW+1])
            for i in range(nW_temp) :
                temp_fit = self.EW(i)
                EW_near[i] = np.flip(np.polyfit(ys, temp_fit, mW))
    
            for j in range(mW+1) :
                tildeW = EW_near[:, j] - w0[j]*(1 - xs_near)**2
                wis_j_init = wis_params[:nW_temp, j]
                wis_j_num, _, ier, _ = fsolve(self.helper_F, wis_j_init, args=(xs_near, tildeW, nW_temp), full_output=1)
                if ier != 1 :
                    print(f"W: fsolve did not converge for nW={nW_temp} and j={j}")
                wis_params[:nW_temp, j] = wis_j_num
    
        w_total = np.empty([nW+1, mW+1])
        w_total[0] = w0
        w_total[1:] = wis_params

        self.W_params = w_total
    
    def compute_K_params(self) :
        # Far Field
        xs = self.xs
        ys = self.ys
        nK = self.ns["K"]
        mK = self.ms["K"]
    
        EK_far = np.empty([2, mK+1])
        for i, r_ind in enumerate((-2, -1)) :
            temp_fit = self.EK(r_ind)
            EK_far[i] = np.flip(np.polyfit(ys, temp_fit, mK))

        x1, x2 = xs[-2], xs[-1]
        p1, p2 = EK_far[0], EK_far[1]
        tildeK = -(p2 / (1 - x2)**2 - p1 / (1 - x1)**2) / (x2 - x1)
        k0     = p1 / (1 - x1)**2 - tildeK*(1 - x1)

        # Near Field
        kis_params = np.zeros([nK, mK+1])

        for nK_temp in range(1, nK+1) :
            xs_near = self.xs[:nK_temp]
            EK_near = np.empty([nK_temp, mK+1])
            for i in range(nK_temp) :
                temp_fit = self.EK(i)
                EK_near[i] = np.flip(np.polyfit(ys, temp_fit, mK))
        
            for j in range(mK+1) :
                tildeK = EK_near[:, j] - k0[j]*(1 - xs_near)**2
                kis_j_init = kis_params[:nK_temp, j]
                kis_j_num, _, ier, _ = fsolve(self.helper_F, kis_j_init, args=(xs_near, tildeK, nK_temp), full_output=1)
                if ier != 1 :
                    print(f"K: fsolve did not converge for nK={nK_temp} and j={j}")
                kis_params[:nK_temp, j] = kis_j_num
    
        k_total = np.empty([nK+1, mK+1])
        k_total[0] = k0
        k_total[1:] = kis_params

        self.K_params = k_total

    def compute_A_params(self) :
        k00 = self.K_params[0, 0]
        
        # Far Field
        xs = self.xs
        ys = self.ys
        nA = self.ns["A"]
        mA = self.ms["A"]
    
        A_far = np.empty([3, mA+1])
        xs_far = xs[-3:]
        for i, r_ind in enumerate((-3, -2, -1)) :
            temp_fit = self.A(r_ind)
            A_far[i] = np.flip(np.polyfit(ys, temp_fit, mA))

        epss = np.zeros([mA+1,])
        a0s = np.zeros([mA+1,])
        
        # Dealing with A0
        nxs = [1-x for x in xs_far]
        sqnxs = [nx*nx for nx in nxs]
        RF0 = [(A_far[i, 0]/xs_far[i] - 1)/nxs[i] for i in range(3)]
        DeltaF1 = (RF0[1] - RF0[0]) / (nxs[1] - nxs[0])
        DeltaX1 = (sqnxs[1] - sqnxs[0]) / (nxs[1] - nxs[0])
        DeltaF2 = (RF0[2] - RF0[1]) / (nxs[2] - nxs[1])
        DeltaX2 = (sqnxs[2] - sqnxs[1]) / (nxs[2] - nxs[1])
        Temp1 = (DeltaF2 - DeltaF1) / (DeltaX2 - DeltaX1)
        Temp2 = DeltaF1 - Temp1*DeltaX1
        Temp3 = RF0[0] - Temp2*nxs[0] - Temp1*sqnxs[0]
        eps0 = -Temp3
        a00 = Temp2 + eps0 - k00
        epss[0] = eps0
        a0s[0] = a00

        # Dealing with Ai
        for j in range(1, mA+1) :
            RF1 = [A_far[i, j]/sqnxs[i] for i in range(3)]
            if j < self.ms["K"]+1 :
                RF1 = [RF1[i] - self.EK_Pade(xs_far[i], j)*nxs[i] - self.K_params[0, j] for i in range(3)]
            DeltaF1 = (RF1[1] - RF1[0])/(nxs[1] - nxs[0])
            DeltaX1 = (sqnxs[1] - sqnxs[0])/(nxs[1] - nxs[0])
            DeltaF2 = (RF1[2] - RF1[1])/(nxs[2] - nxs[1])
            DeltaX2 = (sqnxs[2] - sqnxs[1])/(nxs[2] - nxs[1])
            Temp1 = (DeltaF2 - DeltaF1)/(DeltaX2 - DeltaX1)
            Temp2 = DeltaF1 - Temp1*DeltaX1
            Temp3 = RF1[0] - Temp2*nxs[0] - Temp1*sqnxs[0]
            epsj = Temp3
            aj0 = Temp2
            epss[j] = epsj
            a0s[j] = aj0

        # Near Field
        ais_params = np.zeros([nA, mA+1])

        for nA_temp in range(1, nA+1) :
            xs_near = self.xs[:nA_temp]
            nxs_near = 1 - xs_near
            A_near = np.empty([nA_temp, mA+1])
            for i in range(nA_temp) :
                temp_fit = self.A(i)
                A_near[i] = np.flip(np.polyfit(ys, temp_fit, mA))
    
            # 0th case
            tildeA0 = A_near[:, 0]/xs_near - 1 + eps0*nxs_near - (a00-eps0+k00)*nxs_near**2
            a0_j_init = ais_params[:nA_temp, 0]
            a0_j_num, _, ier, _ = fsolve(self.helper_F, a0_j_init, args=(xs_near, tildeA0, nA_temp), full_output=1)
            if ier != 1 :
                print(f"A: fsolve did not converge for nA={nA_temp} and j=0")
            ais_params[:nA_temp, 0] = a0_j_num

            # Next cases
            for j in range(1, mA+1) :
                tildeA = 0
                if j < self.ms["K"]+1 :
                    tildeA = self.K_params[0, j]*nxs_near**2 + self.EK_Pade(xs_near, j)*nxs_near**3
                tildeA = A_near[:, j] - tildeA - epss[j]*nxs_near**2 - a0s[j]*nxs_near**3
                ais_j_init = ais_params[:nA_temp, j]
                ais_j_num, _, ier, _ = fsolve(self.helper_F, ais_j_init, args=(xs_near, tildeA, nA_temp), full_output=1)
                if ier != 1 :
                    print(f"fsolve did not converge for coefficient index {j} of A")
                    # raise RuntimeError(f"fsolve did not converge for coefficient index {j} of A")
                ais_params[:nA_temp, j] = ais_j_num
    
        a_total = np.empty([nA+2, mA+1])
        a_total[0] = epss
        a_total[1] = a0s
        a_total[2:] = ais_params

        self.A_params = a_total

    def compute_B_params(self) :
        # Far Field
        xs = self.xs
        ys = self.ys
        nB = self.ns["B"]
        mB = self.ms["B"]

        B_far = np.empty([2, mB+1])
        for i, r_ind in enumerate((-2, -1)) :
            temp_fit = np.sqrt(self.B2(r_ind)) - 1
            B_far[i] = np.flip(np.polyfit(ys, temp_fit, mB))

        x1, x2 = xs[-2], xs[-1]
        p1, p2 = B_far[0], B_far[1]
        tildeB = -(p2 / (1 - x2) - p1 / (1 - x1)) / (x2 - x1)
        b0     = p1 / (1 - x1) - tildeB*(1 - x1)

        # Near Field
        bis_params = np.zeros([nB, mB+1])

        for nB_temp in range(1, nB+1) :
            xs_near = self.xs[:nB_temp]
            B_near = np.empty([nB_temp, mB+1])
            for i in range(nB_temp) :
                temp_fit = np.sqrt(self.B2(i)) - 1
                B_near[i] = np.flip(np.polyfit(ys, temp_fit, mB))

        for j in range(mB+1) :
            tildeB = B_near[:, j] - b0[j] * (1 - xs_near)
            bis_j_init = bis_params[:nB_temp, j]
            bis_j_num, _, ier, _ = fsolve(self.helper_B, bis_j_init, args=(xs_near, tildeB, nB_temp), full_output=1)
            if ier != 1 :
                print(f"B: fsolve did not converge for nB={nB_temp} and j={j}")
            bis_params[:nB_temp, j] = bis_j_num
    
        b_total = np.empty([nB+1, mB+1])
        b_total[0] = b0
        b_total[1:] = bis_params

        self.B_params = b_total

    def compute_all_params(self) :
        self.compute_W_params()
        self.compute_K_params()
        self.compute_A_params()
        self.compute_B_params()

        results = {
            "W": self.W_params,
            "K": self.K_params,
            "A": self.A_params,
            "B": self.B_params
        }

        return results

# Example: Kerr Metric

In [3]:
def Sig(r, th) :
    return 1 + (a*np.cos(th)/r)**2

def Kerr_gtt(r, th) :
    return 2*M/(Sig(r, th)*r) - 1

def Kerr_grr(r, th) :
    return Sig(r, th) / (1 - 2*M/r + (a/r)**2)

def Kerr_gpp(r, th) :
    return (r*r + a*a + 2*M*(a*np.sin(th))**2/(Sig(r, th)*r))*np.sin(th)**2

def Kerr_gthth(r, th) :
    return r*r + (a*np.cos(th))**2

def Kerr_gtp(r, th) :
    return -2*M*a*np.sin(th)**2 / (Sig(r, th)*r)

M  = 1.
a  = 0.9
r0 = M + np.sqrt(M*M - a*a)
ns = {"W": 4, "K": 4, "A": 4, "B": 4}
ms = {"W": 2, "K": 2, "A": 2, "B": 2}
g = {}

n_max = ns[max(ns, key=ns.get)]
xs_near = np.arange(0.01, 0.1, 0.01)[:n_max]
xs_far = np.random.uniform(0.999, 1, size=(3,))
xs = np.concatenate((xs_near, xs_far))
rs = r0 / (1 - xs)
ths = np.linspace(0.001, np.pi/2, 100)
R, Th = np.meshgrid(rs, ths, indexing='ij')

g["gtt"] = np.vectorize(Kerr_gtt)(R, Th)
g["grr"] = np.vectorize(Kerr_grr)(R, Th)
g["gpp"] = np.vectorize(Kerr_gpp)(R, Th)
g["gtp"] = np.vectorize(Kerr_gtp)(R, Th)
g["gthth"] = np.vectorize(Kerr_gthth)(R, Th)

parametrizer = KRZParametrizer(g, a, r0, rs, ths, ms, ns)

In [4]:
params = parametrizer.compute_all_params()

print("W params:")
print(params["W"])
print("="*55)
print("K params:")
print(params["K"])
print("="*55)
print("A params:")
print(params["A"])
print("="*55)
print("B params:")
print(params["B"])
print("="*55)

W: fsolve did not converge for nW=3 and j=0
W: fsolve did not converge for nW=3 and j=1
W: fsolve did not converge for nW=3 and j=2
W: fsolve did not converge for nW=4 and j=0
W: fsolve did not converge for nW=4 and j=2
K: fsolve did not converge for nK=4 and j=2
B: fsolve did not converge for nB=4 and j=0
B: fsolve did not converge for nB=4 and j=1
B: fsolve did not converge for nB=4 and j=2
W params:
[[ 8.73032130e-01  1.66602217e-14 -1.26409839e-14]
 [ 1.72359519e-15 -9.98697810e-15  2.24606850e-15]
 [-5.54514902e+02 -5.72149224e+01 -1.41299110e+03]
 [ 1.11435075e+03  6.32139583e+01  1.64227909e+03]
 [ 0.00000000e+00 -1.43623661e+01  0.00000000e+00]]
K params:
[[ 3.92864459e-01 -1.25075976e-07 -6.88295238e-10]
 [-2.79357353e-10 -3.92864333e-01  6.88305174e-10]
 [-9.99982002e-01 -3.92864265e-01 -9.94772812e-01]
 [-8.45858197e-05  3.92865068e-01 -1.41191592e+00]
 [-2.11439756e+01 -2.43410748e-06  2.65791128e+02]]
A params:
[[ 3.92864458e-01  6.79253435e-08  1.94012362e-08]
 [ 1.933022